In [ ]:
import torch
import torch.nn as nn
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx, to_dense_adj

import os
import numpy as np

import networkx as nx

from plotting.plotting_functions import plot_splits

from misc.data_utils_cora import get_splits
from misc.lipschitz_constant import estimate_lipschitz_constant

from models.locally_linear_embeddings import locally_linear_embedding
from models.laplacian_eigenmaps import laplacian_eigemaps
from models.grarep import grarep_train
from models.deepwalk import DeepWalk
from models.logistic_regression import LogisticRegression, train, test

# Data Preperation

In [ ]:
data = Planetoid('./data/Planetoid', name='Cora')[0]

nodes = np.array(range(data.num_nodes))
edges = np.array(data.edge_index.T)

A = to_dense_adj(edge_index=data.edge_index, max_num_nodes=data.num_nodes)[0]

labels = data.y
y_one_hot = torch.nn.functional.one_hot(labels)
y_one_hot = y_one_hot.float()

idx_train, idx_val, idx_test = get_splits(y_one_hot, pct_train=25)

num_classes = 7
input_shape=128

In [ ]:
plot_splits(labels, idx_train, idx_val, idx_test)

# Locally Linear

In [ ]:
embeddings = locally_linear_embedding(A, n_neighbours=5, d=128)
torch.save(embeddings, './embeddings/locally_linear_embeddings.pt')

In [ ]:
embeddings = torch.load('./embeddings/locally_linear_embeddings.pt')

In [ ]:
input_size = embeddings.shape[1]
num_classes = y_one_hot.shape[1]

model = LogisticRegression(input_size, num_classes)

In [ ]:
# Model hyperparameters
lipschitz_constant = estimate_lipschitz_constant(model, embeddings, n_estimates=100000)
learning_rate = 1/(lipschitz_constant*1.2)
print(f'Learning Rate: {learning_rate}')
num_epochs = 20000
display_every = 500
patience = 100
model_path='./model_weights/locally_linear_embedding.pth'
output_path='./history/locally_linear_embeddings_loss.csv'

In [ ]:
# Training
train(model=model, learning_rate=learning_rate, embeddings=embeddings, labels=labels, idx_train=idx_train, idx_val=idx_val, display_every=display_every, num_epochs=num_epochs, model_path=model_path, output_path=output_path)

model.load_state_dict(torch.load(model_path))

# Testing
test(model=model, embeddings=embeddings, labels=labels, idx_test=idx_test)

# Laplacian Eigenmaps

In [ ]:
embeddings = laplacian_eigemaps(A, d=128, verbose=0)
torch.save(embeddings, './embeddings/laplacian_eigenmap_embeddings.pt')

In [ ]:
embeddings = torch.load('./embeddings/laplacian_eigenmap_embeddings.pt')

In [ ]:
input_size = embeddings.shape[1]
num_classes = y_one_hot.shape[1]

model = LogisticRegression(input_size, num_classes)

In [ ]:
# Model hyperparameters
lipschitz_constant = estimate_lipschitz_constant(model, embeddings, n_estimates=10000)
learning_rate = 1/(lipschitz_constant*1.2)
print(f'Learning Rate: {learning_rate}')
num_epochs = 20000
display_every = 100
patience = 20
model_path='./model_weights/laplacian_eigenmaps.pth'

In [ ]:
# Training
train(model=model, learning_rate=learning_rate, embeddings=embeddings, labels=labels, idx_train=idx_train, idx_val=idx_val, num_epochs=num_epochs, model_path=model_path, output_path='./history/laplacian_eigenmaps_loss.csv')
model.load_state_dict(torch.load(model_path))

# Testing
test(model=model, embeddings=embeddings, labels=labels, idx_test=idx_test)

# GraRep Training

In [ ]:
embeddings=grarep_train(A, K=4, emb_size_per_K=32)
torch.save(embeddings, './embeddings/grarep_embedding.pt')

In [ ]:
embeddings = torch.load('./embeddings/grarep_embedding.pt')

In [ ]:
input_size = embeddings.shape[1]
num_classes = y_one_hot.shape[1]

model = LogisticRegression(input_size, num_classes)

In [ ]:
# Model hyperparameters
lipschitz_constant = estimate_lipschitz_constant(model, embeddings, n_estimates=20000)
learning_rate = 1/(lipschitz_constant*1.2)
print(f'Learning Rate: {learning_rate}')
num_epochs = 20000
display_every = 10
patience = 20
model_path='./model_weights/grarep.pth'

In [ ]:
# Training
train(model=model, learning_rate=learning_rate, embeddings=embeddings, labels=labels, idx_train=idx_train, idx_val=idx_val, num_epochs=num_epochs, display_every=display_every, patience=patience, model_path=model_path, output_path='./history/grarep_loss.csv')

model.load_state_dict(torch.load(model_path))

# Testing
test(model=model, embeddings=embeddings, labels=labels, idx_test=idx_test)

# DeepWalk

In [ ]:
G = nx.from_numpy_array(A.numpy())
deepwalk = DeepWalk(G, num_walks=10, length_walk=5, window_size=3, embedding_size=128)
model = deepwalk.train()
embeddings = torch.torch.FloatTensor([model.wv[str(node)] for node in G.nodes()])
torch.save(embeddings, './embeddings/deepwalk_embedding.pt')

In [ ]:
embeddings = torch.load('./embeddings/deepwalk_embedding.pt')

In [ ]:
input_size = embeddings.shape[1]
num_classes = y_one_hot.shape[1]

model = LogisticRegression(input_size, num_classes)

In [ ]:
# Model hyperparameters
lipschitz_constant = estimate_lipschitz_constant(model, embeddings, n_estimates=20000)
learning_rate = 1/(lipschitz_constant*1.2)
print(f'Learning Rate: {learning_rate}')
num_epochs = 20000
display_every = 10
patience = 20
model_path='./model_weights/deepwalk.pth'

In [ ]:
# Training
train(model=model, learning_rate=learning_rate, embeddings=embeddings, labels=labels, idx_train=idx_train, idx_val=idx_val, num_epochs=num_epochs, display_every=display_every, patience=patience, model_path=model_path, output_path='./history/deepwalk_loss.csv')

model.load_state_dict(torch.load(model_path))

# Testing
test(model=model, embeddings=embeddings, labels=labels, idx_test=idx_test)

In [ ]:
import pandas as pd
def get_data(path):

	df = pd.read_csv(path)

	train_loss = df['loss_train'].to_list()
	val_loss = df['loss_val'].to_list()

	return train_loss, val_loss

In [ ]:
import plotly.graph_objects as go

locally_linear_embeddings_train_loss,  locally_linear_embeddings_val_loss =  get_data('./history/locally_linear_embeddings_loss.csv')
laplacian_eigemaps_train_loss,  laplacian_eigemaps_val_loss =  get_data('./history/laplacian_eigenmaps_loss.csv')
grarep_train_loss,  grarep_val_loss =  get_data('./history/grarep_loss.csv')
deepwalk_train_loss, deepwalk_val_loss =  get_data('./history/deepwalk_loss.csv')

x = list(range(1, len(locally_linear_embeddings_train_loss)+1))

trace1 = go.Scatter(x=x, y=locally_linear_embeddings_train_loss, mode='lines', name='Locally Linear Embeddings Train Loss')
trace2 = go.Scatter(x=x, y=laplacian_eigemaps_train_loss, mode='lines', name='Laplacian Eigenmaps Train Loss')
trace3 = go.Scatter(x=x, y=grarep_train_loss, mode='lines', name='Grarep Train Loss')
trace4 = go.Scatter(x=x, y=deepwalk_train_loss, mode='lines', name='Deepwalk Train Loss')

trace5 = go.Scatter(x=x, y=locally_linear_embeddings_val_loss, mode='lines', name='Locally Linear Embeddings Val Loss', line=dict(dash='dash', color='#636EFA'))
trace6 = go.Scatter(x=x, y=laplacian_eigemaps_val_loss, mode='lines', name='Laplacian Eigenmaps Val Loss', line=dict(dash='dash', color='#EF553B'))
trace7 = go.Scatter(x=x, y=grarep_val_loss, mode='lines', name='Grarep Val Loss', line=dict(dash='dash', color='#00CC96'))
trace8 = go.Scatter(x=x, y=deepwalk_val_loss, mode='lines', name='Deepwalk Val Loss', line=dict(dash='dash', color='#AB63FA'))

fig = go.Figure()

fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_trace(trace3)
fig.add_trace(trace4)
fig.add_trace(trace5)
fig.add_trace(trace6)
fig.add_trace(trace7)
fig.add_trace(trace8)

fig.update_layout(
    title="Shallow Embedding Methods Loss",
    xaxis_title="Epoch",
    yaxis_title="Loss",
    legend_title="Loss",
    template="plotly"
)

fig.update_xaxes(range=[0, 100])
fig.update_yaxes(range=[0.4, 2])

fig.show()